In [87]:
%run gower_function-v6.4.ipynb
from pymongo import MongoClient
from random import randint
import pandas as pd
import numpy as np
import re
import category_encoders as ce
from scipy.spatial.distance import pdist,squareform, jaccard, cosine
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

    age gender civil_status   salary  has_children  available_credit
0  21.0      M      MARRIED   3000.0           1.0            2200.0
1  21.0      M       SINGLE   1200.0           0.0             100.0
2  19.0      N       SINGLE  32000.0           1.0           22000.0
3  30.0      M       SINGLE   1800.0           1.0            1100.0
4  21.0      F      MARRIED   2900.0           1.0            2000.0
5  21.0      F       SINGLE   1100.0           0.0             100.0
6  19.0      F        WIDOW  10000.0           0.0            6000.0
7  30.0      F     DIVORCED   1500.0           1.0            2200.0
8   NaN   None         None      NaN           NaN               NaN
[False  True  True False False False]
[21.0 3000.0 1.0 2200.0]
['M' 'MARRIED']
[21.0 21.0 19.0 30.0 21.0 21.0 19.0 30.0 nan]
[3000.0 1200.0 32000.0 1800.0 2900.0 1100.0 10000.0 1500.0 nan]
[1.0 0.0 1.0 1.0 1.0 0.0 0.0 1.0 nan]
[2200.0 100.0 22000.0 1100.0 2000.0 100.0 6000.0 2200.0 nan]
[[0.         0.3590238

gower_function-v6.4.ipynb:71: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  "source": [
gower_function-v6.4.ipynb:71: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  "source": [
gower_function-v6.4.ipynb:71: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  "source": [
gower_function-v6.4.ipynb:71: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  "source": [
gower_function-v6.4.ipynb:71: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.s

In [39]:
client = MongoClient(port=27017)
db = client.weare    

In [40]:
student_queries = db.students.find({})
students = pd.DataFrame(list(student_queries))

In [41]:
students['id']=students.Email.str.split("@", n=1, expand=True)[0]
if '_id' in students.columns:
    del students['_id']

In [42]:
#drop rows with 5 or more null values
students.dropna(thresh=5, inplace=True)
students = students.replace('', np.nan)

In [43]:
#number of courses, credits are too dirty as many entered text instead of a number
drop_columns = ["surveyCompletion", "Duration (in seconds)", "NumCourses", "transferCredits", "NumTranCredits",
                "WhyProfile", "otherChannels", "otherEmploy", "NonUS", "otherIndusry", "KnownThroughProfile",
                "otherEth", "PPLinPerson", "otherEmail"]
#only drop these columns if they actually exist in the data
drop_columns = [col for col in drop_columns if col in students.columns]
students.drop(columns=drop_columns, inplace=True)
#names of all the soc columns
SOCs = ['SOC'+str(x+1) for x in range(10)]
#names of all the CCE columns
CCEs = ['CCE'+str(x+1) for x in range(24)]
#mapping from each text column to ordinal values
#format is array of dicts containing keys 'col' and 'mapping'
ordinal_cols_mapping = []

In [44]:
#the following are mappings from textual column values to numberical ones
important_scale = [
        ('Extremely important', 7),
        ('Pretty important', 6),
        ('Fairly important', 5),
        ('Moderately important', 4),
        ('Somewhat important', 3),
        ('A little important', 2),
        ('Not at all important', 1)
]

interest_scale = [
    ('Extremely interested', 5),
    ('Rather interested', 4),
    ('Somewhat interested', 3),
    ('A bit of interest', 2),
    ('Not at all interested', 1)
]

Agree_scale = [
        ('Strongly agree', 5),
        ('Somewhat agree', 4),
        ('Neither agree nor disagree', 3),
        ('Somewhat disagree', 2),
        ('Strongly disagree', 1)
]

In [45]:
#fill in the mapping array 
for SOC in SOCs:
    ordinal_cols_mapping.append({
        "col": SOC,
        "mapping": Agree_scale
    })

for CCE in CCEs:
    ordinal_cols_mapping.append({
        "col": CCE,
        "mapping": Agree_scale
    })

for PeerInfo in ["PeerAvail", "PeerProfession", "PeerEdu", "PeerDetails"]:
    ordinal_cols_mapping.append({
        "col": PeerInfo,
        "mapping": important_scale
    })

ordinal_cols_mapping.append({
    "col": "Mconnected",
    "mapping": interest_scale
})

In [46]:
students.gender

0      Female
1        Male
2      Female
3      Female
4      Female
5        Male
6        Male
7        Male
8      Female
9      Female
10     Female
11       Male
12       Male
13     Female
14       Male
15       Male
16     Female
17     Female
18     Female
19     Female
20     Female
21     Female
22     Female
23       Male
24       Male
25       Male
26     Female
27     Female
28       Male
29     Female
        ...  
351       NaN
352       NaN
353       NaN
354       NaN
355       NaN
356       NaN
357       NaN
358       NaN
359       NaN
360       NaN
361       NaN
362       NaN
363       NaN
364       NaN
365       NaN
366       NaN
367       NaN
368       NaN
369       NaN
370       NaN
371       NaN
372       NaN
373       NaN
374       NaN
375       NaN
376       NaN
377       NaN
378       NaN
379       NaN
380       NaN
Name: gender, Length: 381, dtype: object

In [47]:
encoder = ce.OrdinalEncoder(mapping=ordinal_cols_mapping, return_df=True)
df = encoder.fit_transform(students)
#Create new columns by taking the mean of related columns for each row
df['SOC_f'] = df.apply(lambda row: np.mean([row['SOC9'], row['SOC10']]), axis=1)
df['SOC_id'] = df.apply(lambda row: np.mean([row['SOC1'], row['SOC2'], row['SOC3'], row['SOC4'], row['SOC5']]), axis=1)
df['CCE_IR'] = df.apply(lambda row: np.mean([row['CCE13'], row['CCE10'], row['CCE19'], row['CCE9']]), axis=1)
df['CCE_Coor'] = df.apply(lambda row: np.mean([row['CCE24'], row['CCE20'], row['CCE22'], row['CCE21'], row['CCE17']]), axis=1)
df['CCE_SS'] = df.apply(lambda row: np.mean([row['CCE1'], row['CCE2'], row['CCE3'], row['CCE4']]), axis=1)
df.drop(columns=CCEs + SOCs, inplace=True)

In [48]:
none_i = re.compile(r'none', flags=re.IGNORECASE)
df['kids'].replace(none_i, 0, inplace=True)
none_i = re.compile(r'zero', flags=re.IGNORECASE)
df['kids'].replace(none_i, 0, inplace=True)
stringany = re.compile(r'[a-zA-Z ()]+', flags=re.IGNORECASE)
print(f'Before digit processing, kids are {df.kids.unique()}')
df.kids = df['kids'].replace(stringany, '')

Before digit processing, kids are [ 0.  1.  2.  3.  4. nan  5.  9.]


In [49]:
categorical_cols = ["gender", "InUS", "ethnicity", "Usstate", "marrital", "employment", "industry"]
categorical_cols = [c for c in categorical_cols if c in df.columns]
df_c_mode = df[categorical_cols].mode()
print(f'mode listed are \n{df_c_mode.iloc[0]}')
print(len(df_c_mode))
# df.loc[:, categorical_cols].fillna(df_c_mode.iloc[0], inplace=True)
for col in categorical_cols+['kids']:
    df[col].fillna(df[col].mode().iloc[0], inplace=True)
#onehotecoder = ce.OneHotEncoder(cols=categorical_cols, handle_unknown='ignore')
#df = onehotecoder.fit_transform(df)

mode listed are 
gender                         Female
InUS                              Yes
ethnicity                   Caucasian
Usstate                  Pennsylvania
marrital                      Married
employment    Working (paid employee)
industry         Educational services
Name: 0, dtype: object
1


In [50]:
#col_non_num = [c for c in df.columns if df[c].dtype == 'object']
#df.drop(columns=col_non_num, inplace=True)

In [51]:
# fill with mode, mean, or median
df_mode, df_mean, df_median = df.mode().iloc[0], df.mean(), df.median()
df.fillna(df_median, inplace=True)
#data = StandardScaler().fit_transform(df.values)

In [52]:
#A_sparse = sparse.csr_matrix(data)
#similarities = cosine_similarity(A_sparse)

In [53]:
#similarities_sparse = cosine_similarity(A_sparse, dense_output=False)
#euclidean_matrix = pd.DataFrame(squareform(pdist(data, metric='euclidean')), columns=students['id'], index=students['id'])

In [54]:
#cosine_matrix = pd.DataFrame(squareform(pdist(data, metric='cosine')), columns=students['id'], index=students['id'])
#cos = cosine_matrix.to_dict(orient='records')  # Here's our added param..
#db.cosine_matrix.insert_many(cos)
#euc = euclidean_matrix.to_dict(orient='records2q34')  # Here's our added param..
#db.euclidean_matrix.insert_many(euc)

In [90]:
df.ActiveDuty

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9      NaN
10     NaN
11     NaN
12     NaN
13     NaN
14     NaN
15     NaN
16     NaN
17     NaN
18     NaN
19     NaN
20     NaN
21     NaN
22     NaN
23     NaN
24     NaN
25     NaN
26     NaN
27     NaN
28     NaN
29     NaN
      ... 
351    NaN
352    NaN
353    NaN
354    NaN
355    NaN
356    NaN
357    NaN
358    NaN
359    NaN
360    NaN
361    NaN
362    NaN
363    NaN
364    NaN
365    NaN
366    NaN
367    NaN
368    NaN
369    NaN
370    NaN
371    NaN
372    NaN
373    NaN
374    NaN
375    NaN
376    NaN
377    NaN
378    NaN
379    NaN
380    NaN
Name: ActiveDuty, Length: 381, dtype: object

In [55]:
list(df.columns)

['ActiveDuty',
 'BWS',
 'Degree',
 'Email',
 'First Name',
 'IP Address',
 'InUS',
 'Last Name',
 'Latitude',
 'Longitude',
 'Major',
 'Mconnected',
 'Military',
 'OCEnabler',
 'PeerAvail',
 'PeerDetails',
 'PeerEdu',
 'PeerProfession',
 'ProfileComplete',
 'Response ID',
 'Usstate',
 'Visit',
 'age',
 'employment',
 'enrollYear',
 'ethnicity',
 'gender',
 'grade',
 'industry',
 'kids',
 'marrital',
 'progress',
 'id',
 'SOC_f',
 'SOC_id',
 'CCE_IR',
 'CCE_Coor',
 'CCE_SS']

In [56]:
#from scipy.spatial.distance import cosine
#from sklearn.metrics import jaccard_similarity_score

In [57]:
x = [1, 0, 0, 1, 0, 0]
y = [1, 0, 1, 0, 0, 0]
print(1 - cosine(x,y))
print(jaccard_similarity_score(x,y))
D = gower_distances(df)
n

0.5


NameError: name 'jaccard_similarity_score' is not defined

In [101]:
numeric_columns = df._get_numeric_data().columns
col_is_categorical = [col not in numeric_columns for col in df.columns]
D = gower_distances(df, categorical_features=col_is_categorical)

/home/matt/.conda/envs/weare/lib/python3.7/site-packages/ipykernel_launcher.py:71: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.


[ True  True  True  True  True  True  True  True False False  True False
  True  True False False False False False  True  True  True False  True
 False  True  True  True  True False  True False  True False False False
 False False]
[40.35139465 -75.93060303 3 7 1 5 4 40.0 32.0 2017.0 0.0 14.0 2.5 5.0 4.5
 3.8 3.5]
[nan 'No' 'Master degree' 'sxa459@psu.edu' 'Sana' '72.94.219.150' 'Yes'
 'Abdulateef' 'Information Sciences and Tech' 'No' nan 'R_11aRu8dfJXw0Fww'
 'Pennsylvania' 'No' 'Not working (looking for work)' 'Caucasian' 'Female'
 'Freshman' 'Educational services' 'Married' 'sxa459']
[40.35139465 40.10319519 40.78489685 38.29069519 40.80769348 39.95530701
 38.81750488 40.73910522 32.48139954 40.292099 40.79750061 40.26669312
 39.89419556 38.92660522 28.48069763 40.78489685 39.9683075 40.51950073
 33.02470398 40.93539429 32.77870178 40.79750061 42.19729614 40.55050659
 40.85969543 38.83180237 39.91929626 40.44949341 40.78489685 42.20080566
 27.32550049 37.42500305 40.78489685 32.5702

In [141]:
from IPython.display import display
def get_match(index, D, df):
    best_score = sorted(D[index])[1]
    best_entry = list(D[index]).index(best_score)
    og_user = df.iloc[[index]]
    best_user = df.iloc[[best_entry]]
    concat = pd.concat([og_user, best_user])
    return concat
def display_matches(D, df, num_matches=10):
    frame = get_match(0, D, df)
    for i in range(1, num_matches):
        match = get_match(i, D, df)
        frame = pd.concat([frame, match])
    return frame

In [140]:
display

,ActiveDuty,BWS,Degree,Email,First Name,IP Address,InUS,Last Name,Latitude,Longitude,...,industry,kids,marrital,progress,id,SOC_f,SOC_id,CCE_IR,CCE_Coor,CCE_SS
3,NaN,Yes,Bachelor degree,ama5044@psu.edu,Adrian,72.9.8.222,Yes,Alfano,38.290695,-76.522102,...,Management of companies or enterprises,0.0,Married,6.0,ama5044,1.5,4.0,4.50,3.8,4.5
311,NaN,No,Bachelor degree,rmb80@psu.edu,Rachel,24.210.98.97,Yes,Bemis,42.066696,-80.166397,...,Lodging or food services,0.0,Single/Never Married,20.0,rmb80,1.0,3.4,4.75,3.4,3.0


In [123]:
df.iloc[[172]]


,ActiveDuty,BWS,Degree,Email,First Name,IP Address,InUS,Last Name,Latitude,Longitude,...,industry,kids,marrital,progress,id,SOC_f,SOC_id,CCE_IR,CCE_Coor,CCE_SS
172,NaN,No,Master degree,deb77@psu.edu,Debra,73.187.239.247,Yes,Barshinger,39.893906,-76.557999,...,Educational services,0.0,Married,8.0,deb77,3.0,4.4,4.25,4.4,4.25


In [79]:
df.Latitude.unique()

array([ 40.35139465,  40.10319519,  40.78489685,  38.29069519,
        40.80769348,  39.95530701,  38.81750488,  40.73910522,
        32.48139954,  40.292099  ,  40.79750061,  40.26669312,
        39.89419556,  38.92660522,  28.48069763,  39.9683075 ,
        40.51950073,  33.02470398,  40.93539429,  32.77870178,
        42.19729614,  40.55050659,  40.85969543,  38.83180237,
        39.91929626,  40.44949341,  42.20080566,  27.32550049,
        37.42500305,  32.57020569,  41.14399719,  40.38999939,
        41.88250732,  40.99780273,  47.55130005,  33.52070618,
        31.13450623,  47.57099915,  47.90150452,  40.01919556,
        44.97290039,  39.08399963,  26.53329468,  33.78030396,
        35.8874054 ,  33.86329651,  39.82679749,  35.44360352,
        35.18330383,  41.6269989 ,  40.76939392,  34.30670166,
        41.3572998 ,  42.11669922,  42.13519287,  40.31489563,
        35.07850647,  39.42379761,  45.67990112,  40.18080139,
        40.11720276,  40.05929565,  42.66099548,  41.48